In [2]:
import requests
from bs4 import BeautifulSoup
import re

# URL to scrape
#url = 'https://www.espncricinfo.com/records/year/team-match-results/2021-2021/twenty20-internationals-3?team=6'
#url = 'https://www.espncricinfo.com/records/year/team-match-results/2022-2022/twenty20-internationals-3?team=6'
url = 'https://www.espncricinfo.com/records/year/team-match-results/2023-2023/twenty20-internationals-3?team=6'


# Fetch the content from the URL
response = requests.get(url)
html_content = response.text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all links in the scorecard section
scorecard_links = soup.find_all('a', href=True)

# Initialize lists to store series IDs and match IDs
series_ids = []
match_ids = []

# Extract series ID and match ID from each link
for link in scorecard_links:
    href_value = link['href']
    if '/series/' in href_value:
        # Find all 7-digit integers in the href value
        ids = re.findall(r'\d{7}', href_value)
        if len(ids) >= 2:
            series_id = ids[0]
            match_id = ids[1]
            series_ids.append(series_id)
            match_ids.append(match_id)

# Print the lists
print('Series IDs:', series_ids)
print('Match IDs:', match_ids)


Series IDs: ['1387592', '1387592', '1389381', '1389381', '1389381', '1389381', '1389381', '1398685', '1398685', '1398685', '1384631', '1384631', '1381201', '1381201', '1381201', '1381201', '1381201', '1348633', '1348633', '1348633', '1348629', '1348629', '1348629', '1388181', '1412545', '1388181']
Match IDs: ['1387599', '1387598', '1389395', '1389394', '1389393', '1389392', '1389391', '1399120', '1399117', '1399113', '1384637', '1384634', '1381221', '1381220', '1381219', '1381218', '1381217', '1348651', '1348650', '1348649', '1348642', '1348641', '1348640', '1388215', '1412550', '1388214']


In [1]:
import requests

def getMatchData(seriesId, matchId):
    url = "https://hs-consumer-api.espncricinfo.com/v1/pages/match/overs/details"
    params = {
        'lang': 'en',
        'seriesId': seriesId,
        'matchId': matchId,
        'mode': 'ALL',
    }
    headers = {
        'accept': 'application/json',
        'User-Agent': 'Your User Agent',  # Set an appropriate user agent

    }
    response = requests.get(url, params=params, headers=headers, verify=False, allow_redirects=True)
    return response.json()

def getMatchInfo(seriesId, matchId):
    url = "https://hs-consumer-api.espncricinfo.com/v1/pages/match/home?"
    params = {
        'lang': 'en',
        'seriesId': seriesId,
        'matchId': matchId,
        #'mode': 'ALL',
    }
    headers = {
        'accept': 'application/json',
        'User-Agent': 'Your User Agent',  # Set an appropriate user agent

    }
    response = requests.get(url, params=params, headers=headers, verify=False, allow_redirects=True)
    return response.json()

In [ ]:
import pandas as pd 

############INNINGS 1#########

# Initialize an empty dictionary to store the results
rows = []

# Loop through each pair of series ID and match ID
for series_id, match_id in zip(series_ids, match_ids):

    # Replace the second ID in the original code with the current match ID
    full_data = getMatchData(series_id, match_id)
    
    # Check if the data for the current ID is available
    if full_data:
        
        
        # Check if predictions exist for the first innings
        if 'inningOvers' in full_data and full_data['inningOvers'] and full_data['inningOvers'] is not None:
            try:
                predictions = full_data['inningOvers'][0]['stats']
        
                
                # Extract the predictions for indices 1 to 20
                for i in range(len(predictions)):
                    
                    # Use a try-except block to catch IndexError
                    try:
                        #if 'predictions' in predictions[i] and predictions[i]['predictions'] is not None:
                        if predictions[i]['balls'] is not None:
                            
                            balls = predictions[i]['balls']
                            
                            for j in range(len(balls)):
                                # Use a nested try-except block to catch IndexError
                                try:
                                    bowler_stat = balls[j].get('bowlerStatText', 'N/A')
                                    batsmanStatText = balls[j].get('batsmanStatText', 'N/A')
                                    pitchline1 = balls[j].get('pitchLine', 'N/A')
                                    pitchlength1 = balls[j].get('pitchLength', 'N/A')
                                    dismissalType = balls[j].get('dismissalType', 'N/A')
                                    bowlerPlayerId = balls[j].get('bowlerPlayerId', 'N/A')
                                    batsmanPlayerId = balls[j].get('batsmanPlayerId', 'N/A'),
                                    batsmanRuns = balls[j].get('batsmanRuns', 'N/A'),
                                    wagonX = balls[j].get('wagonX', 'N/A'),
                                    wagonY = balls[j].get('wagonY', 'N/A'),
                                    wagonZone = balls[j].get('wagonZone', 'N/A')
                                    shotType = balls[j].get('shotType', 'N/A')
                                    shotControl = balls[j].get('shotControl', 'N/A')
                                    



                                    row = {
                                        'matchid': match_id,
                                        'seriesid': series_id,
                                        'over': i,
                                        'ball': j,
                                        'bowler_stat': bowler_stat,
                                        'batsmanStatText': batsmanStatText,
                                        'pitchline1': pitchline1,
                                        'pitchlength1': pitchlength1,
                                        'dismissalType': dismissalType,
                                        'bowlerPlayerId': bowlerPlayerId,
                                        'batsmanPlayerId': batsmanPlayerId,
                                        'batsmanRuns': batsmanRuns,
                                        'wagonY': wagonY,
                                        'wagonX': wagonX,
                                        'wagonZone': wagonZone,
                                        'shotType': shotType,
                                        'shotControl': shotControl
                                    }
                                    rows.append(row)
                                except IndexError:
                                    # Skip if there's no data for the current ball
                                    continue
                    except IndexError:
                        # Skip if there's no data for the current prediction
                        print(f"No predictions available for index {i} of ID {id}.")
                        continue
            except IndexError:
                # Skip if there's no data for the second element of 'inningOvers'
                print(f"No inning data available for ID {id}.")
        else:
            print(f"Skipping ID {id} as no data is available.")


       # Store the predictions in the results dictionary

BBL_Mens_2223_1stinnings = pd.DataFrame(rows)       
#tnpl_2ndinnings = pd.DataFrame(rows)                 
# Now you can access the predictions for each ID and index
#for key, second_innings in results_by_id.items():
#    print(f"Predictions for {key}: {second_innings}")


In [ ]:
import pandas as pd 

############INNINGS 2#########

# Initialize an empty dictionary to store the results
rows = []

# Loop through each pair of series ID and match ID
for series_id, match_id in zip(series_ids, match_ids):

    # Replace the second ID in the original code with the current match ID
    full_data = getMatchData(series_id, match_id)
    
    
    # Check if the data for the current ID is available
    if full_data:
        
        # Check if predictions exist for the first innings
        if 'inningOvers' in full_data and full_data['inningOvers'] and full_data['inningOvers'] is not None:
            try:
                predictions = full_data['inningOvers'][1]['stats']
                
                # Extract the predictions for indices 1 to 20
                for i in range(len(predictions)):
                    
                    # Use a try-except block to catch IndexError
                    try:
                        #if 'predictions' in predictions[i] and predictions[i]['predictions'] is not None:
                        if predictions[i]['balls'] is not None:
                            
                            balls = predictions[i]['balls']
                            
                            for j in range(len(balls)):
                                # Use a nested try-except block to catch IndexError
                                try:
                                    bowler_stat = balls[j].get('bowlerStatText', 'N/A')
                                    batsmanStatText = balls[j].get('batsmanStatText', 'N/A')
                                    pitchline1 = balls[j].get('pitchLine', 'N/A')
                                    pitchlength1 = balls[j].get('pitchLength', 'N/A')
                                    dismissalType = balls[j].get('dismissalType', 'N/A')
                                    bowlerPlayerId = balls[j].get('bowlerPlayerId', 'N/A')
                                    batsmanPlayerId = balls[j].get('batsmanPlayerId', 'N/A'),
                                    batsmanRuns = balls[j].get('batsmanRuns', 'N/A'),
                                    wagonX = balls[j].get('wagonX', 'N/A'),
                                    wagonY = balls[j].get('wagonY', 'N/A'),
                                    wagonZone = balls[j].get('wagonZone', 'N/A')
                                    shotType = balls[j].get('shotType', 'N/A')
                                    shotControl = balls[j].get('shotControl', 'N/A')
                                    



                                    row = {
                                        'matchid': match_id,
                                        'seriesid': series_id,
                                        'over': i,
                                        'ball': j,
                                        'bowler_stat': bowler_stat,
                                        'batsmanStatText': batsmanStatText,
                                        'pitchline1': pitchline1,
                                        'pitchlength1': pitchlength1,
                                        'dismissalType': dismissalType,
                                        'bowlerPlayerId': bowlerPlayerId,
                                        'batsmanPlayerId': batsmanPlayerId,
                                        'batsmanRuns': batsmanRuns,
                                        'wagonY': wagonY,
                                        'wagonX': wagonX,
                                        'wagonZone': wagonZone,
                                        'shotType': shotType,
                                        'shotControl': shotControl
                                    }
                                    rows.append(row)
                                except IndexError:
                                    # Skip if there's no data for the current ball
                                    continue
                    except IndexError:
                        # Skip if there's no data for the current prediction
                        print(f"No predictions available for index {i} of ID {id}.")
                        continue
            except IndexError:
                # Skip if there's no data for the second element of 'inningOvers'
                print(f"No inning data available for ID {id}.")
        else:
            print(f"Skipping ID {id} as no data is available.")


       # Store the predictions in the results dictionary

#tnpl_1stinnings_2024 = pd.DataFrame(rows)       
BBL_Mens_2223_2ndinnings = pd.DataFrame(rows)                 
# Now you can access the predictions for each ID and index
#for key, second_innings in results_by_id.items():
#    print(f"Predictions for {key}: {second_innings}")


In [ ]:
combined_df = pd.concat([BBL_Mens_2223_1stinnings, BBL_Mens_2223_2ndinnings], ignore_index=True)
combined_df['batsmanPlayerId'] = combined_df['batsmanPlayerId'].apply(lambda x: x[0])
combined_df['batsmanRuns'] = combined_df['batsmanRuns'].apply(lambda x: x[0])
combined_df['wagonY'] = combined_df['wagonY'].apply(lambda x: x[0])
combined_df['wagonX'] = combined_df['wagonX'].apply(lambda x: x[0])# First, merge DF1 and DF2 on 'Name' and 'Batting Style'





In [ ]:
combined_df.to_csv('C:\\Users\\kripa\\Desktop/T20I_Mens_2023.csv', index=False)